# **파이썬 머신러닝 완벽가이드 ch9 pg. 562-579**

### **01. 추천시스템의 개요와 배경**
- 추천시스템에 사용되는 데이터  
  -구매한 상품  
  -장바구니에 추가한 상품  
  -평가한 영화 평점/제품 평가  
  -스스로 작성한 자신의 취향  
  -사용자가 클릭한 것  

- 추천 시스템의 유형  
  -콘텐츠 기반 필터링 방식  
  -협업 필터링 방식  
     - 최근접 이웃 협업 필터링  
     - 잠재 요인 협업 필터링: 많이 사용

- 개인화 특성을 좀 더 강화하기 위해 하이브리드 형식으로 콘텐츠 기반과 협업기반 적절히 결합해 사용

### **02. 콘텐츠 기반 필터링 추천 시스템**
- 선호하는 특정한 아이템과 비슷한 아이템 추천  
(ex) 영화: 그 영화의 장르, 출연 배우, 감독, 영화 키워드 등의 콘텐츠와 유사한 다른 영화 추천

### **03. 최근접 이웃 협업 필터링**
- 평점 정보, 상품 구매 이력 등 사용자 행동 양식만을 기반으로 추천 수행
- 목표: 사용자-아이템 평점 매트릭스와 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측 평가
- 사용자-아이템 평점 매트릭스는 희소 행렬
- **최근접 이웃 방식**(메모리 협업 필터링)
  - 사용자 기반: 당신과 비슷한 고객들이 다음 상품도 구매했습니다.
  - 아이템 기반: 이 상품을 선택한 다른 고객들은 다음 상품도 구매했습니다.  
    -아이템의 속성과 상관없이 사용자들이 그 아이템을 좋아하는지/싫어하는지의 평가 척도가 유사한 아이템 추천
  - 일반적으로 아이템 기반 협업 필터링이 정확도 더 높음
  - 코사인 유사도 많이 사용





### **04. 잠재 요인 협업 필터링**
- 잠재 요인 추출해 추천 예측
- 행렬 분해: 대규모 다차원 행렬을 SVD와 같은 차원 감소 기법으로 분해하는 과정에서 잠재 요인 추출
- '잠재 요인'을 기반으로 사용자-아이템 행렬 데이터를 사용자-잠재요인 행렬/아이템-잠재요인 행렬의 전치 행렬로 분해
 -> 내적을 통해 새로운 예측 사용자-아이템 평점 행렬 데이터를 만들어서 사용자가 아직 평점을 부여하지 않은 아이템에 대한 예측 평점을 생성하는 것

- **행렬 분해**  
  -SVD, NMF 등의 방식 이용  
  -주로 SVD 방식 이용하지만, NaN 값이 없는 행렬에만 적용할 수 있음  
  -NaN값이 있다면, 확률적 경사 하강법(SGD) 이용해 SVD 수행
    - P와 Q 행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q 유추



In [4]:
import numpy as np

#원본 행렬
R=np.array([[4,np.NaN,np.NaN,2,np.NaN],
            [np.NaN,5,np.NaN,3,1],
            [np.NaN,np.NaN,3,4,4],
            [5,2,1,2,np.NaN]])
num_users,num_items=R.shape
K=3

np.random.seed(1)
P=np.random.normal(scale=1./K,size=(num_users,K))
Q=np.random.normal(scale=1./K,size=(num_items,K))

In [6]:
#실제 R 행렬과 예측 행렬의 오차행렬의 오차 구하기
from sklearn.metrics import mean_squared_error

def get_rmse(R,P,Q,non_zeros):
  error=0
  full_pred_matrix=np.dot(P,Q.T)

  x_non_zero_ind=[non_zero[0] for non_zero in non_zeros]
  y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
  R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
  full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
  mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
  rmse = np.sqrt(mse)
    
  return rmse

In [7]:
# SGD 기반으로 행렬분해 수행 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=10000
learning_rate=0.01
r_lambda=0.01

# 업데이트
for step in range(steps): 
    for i, j, r in non_zeros:
        
        eij = r - np.dot(P[i, :], Q[j, :].T)
        
        P[i,:] = P[i,:] + learning_rate * ( eij * Q[j,:] - r_lambda*P[i,:] )
        Q[j,:] = Q[j,:] + learning_rate * ( eij * P[i,:] - r_lambda*Q[j,:] )

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
      print("### iteration step : ",step,"rmse : ",rmse)

### iteration step :  0 rmse :  3.2388050277987723
### iteration step :  50 rmse :  0.4876723101369648
### iteration step :  100 rmse :  0.1564340384819247
### iteration step :  150 rmse :  0.07455141311978046
### iteration step :  200 rmse :  0.04325226798579314
### iteration step :  250 rmse :  0.029248328780878973
### iteration step :  300 rmse :  0.022621116143829466
### iteration step :  350 rmse :  0.019493636196525135
### iteration step :  400 rmse :  0.018022719092132704
### iteration step :  450 rmse :  0.01731968595344266
### iteration step :  500 rmse :  0.016973657887570753
### iteration step :  550 rmse :  0.016796804595895633
### iteration step :  600 rmse :  0.01670132290188466
### iteration step :  650 rmse :  0.01664473691247669
### iteration step :  700 rmse :  0.016605910068210026
### iteration step :  750 rmse :  0.016574200475705
### iteration step :  800 rmse :  0.01654431582921597
### iteration step :  850 rmse :  0.01651375177473524
### iteration step :  900 rms

In [8]:
pred_matrix=np.dot(P,Q.T)
print('예측 행렬:\n',np.round(pred_matrix,3))

예측 행렬:
 [[3.991 1.951 1.108 1.998 1.569]
 [4.23  4.978 1.074 2.987 1.005]
 [5.028 2.487 2.988 3.98  3.985]
 [4.974 2.002 1.003 2.002 1.555]]
